In [10]:
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

# 计算欧式距离，即两点间的直线距离 
#参数：vector1-List列表，n维属性坐标值构成的向量
#      vector2-List列表，n维属性坐标值构成的向量
#返回值：浮点数，欧式距离
def euclDistance(vector1, vector2):  
    return np.sqrt(np.sum(np.power(vector2 - vector1, 2)))  

# 创建一个数据集，包含2个类别共8个样本
def createDataSet():
# 生成一个矩阵，每行表示一个样本
    group = np.array([[1.0,0.9], [1.0,1.0], [0.8,0.9], [0.6,0.65],
    [0.1, 0.2], [0.3,0.4], [0.2,0.3], [0.0, 0.1]])
# 监督学习，手工设置8个样本所属的类别标签
    labels = ['A', 'A','A', 'A', 'B', 'B','B', 'B']
    return group, labels

# KNN分类算法函数实现
#参数：newInput-List列表，待分类的数据点
#      dataSet-List列表，已分类点坐标
#      lables-List列表，分类标签
#      k-整数，近邻数量
#返回值：maxIndex-字符，分类结果
def kNNClassify(newInput, dataSet, labels, k):
    numSamples = dataSet.shape[0]   # shape[0]表示行数
    distance = []
#计算newInput与dataSet中个点的距离，放入distance列表内
    for vec in dataSet:
        distance.append(euclDistance(newInput,vec))        
#对距离排序
    sortedDistIndices = np.argsort(distance)
    classCount = {}
#选择k个最近邻
    for i in range(k):
        voteLabel = labels[sortedDistIndices[i]]
        classCount[voteLabel] = classCount.get(voteLabel, 0) + 1
    maxCount = 0
    for key, value in classCount.items():
        if value > maxCount:
            maxCount = value
            maxIndex = key
    return maxIndex

# 生成数据集和类别标签
dataSet, labels = createDataSet()
#K取值3，调用K近邻分类算法
k = 3
#对testX进行分类
testX = np.array([1.2, 1.0])
outputLabel = kNNClassify(testX, dataSet, labels, 3)
print( "Your input is:", testX, "and classified to class: ", outputLabel)
#对testY进行分类
testY = np.array([0.1, 0.3])
outputLabel = kNNClassify(testY, dataSet, labels, 3)
print("Your input is:", testY, "and classified to class: ", outputLabel)

Your input is: [1.2 1. ] and classified to class:  A
Your input is: [0.1 0.3] and classified to class:  B


In [15]:
import numpy as np
import pandas as pd

#读取excel文件，将原来二值表格转换为2维列表，用于后续处理
#参数：fname-excel文件名
#返回值：data2Dlist-2维列表
def loadExcel(fname):
    data2Dlist=[]
    pd1 = pd.read_excel(fname,'Sheet1')
    print('读入的数据文件首5行：')
    print(pd1.head())
    for index ,row in pd1.iterrows():
        list1 = row.tolist()
        list1.pop(0)
        data2Dlist.append(list1)
#显示2维列表前5项
    print('转化后的2维列表前5项：')
    print(data2Dlist[:4])
#返回该2维列表
    return data2Dlist

BMData=loadExcel('Section4-2 bankMarketing_info.xlsx')
BMLabel= ['工龄', '工作性质', '教育程度', '婚姻状况']
print(BMLabel)

#计算样本数据集分类标签为Y/N的概率
#参数：dataset-List列表，训练集，包含了样本数据和分类结果
#      cls_val-字符，取值为Y/N
#返回值：cnt/len(data)-浮点数，训练集中分类标签为cls_vla的频率
def Prob(dataset,cls_val):
    cnt = 0.0
    for e in dataset:
#每个样本的第4列为分类标签结果
        if e[4] == cls_val:
            cnt += 1
    return cnt/len(dataset)

#计算条件概率
#参数：dataset-List列表，训练集，包含了样本数据和分类结果
#      cls_val-字符，取值为Y/N
#      attr_index-整数，属性序号
#      attr_val-整数，属性取值
#返回值：cnt2/cnt1-浮点数，条件概率：属性取值|分类结果
def ConditionP(dataset,cls_val,attr_index,attr_val):
    cnt1 = 0.0
    cnt2 = 0.0
    for e in dataset:
#每个样本的第4列为分类标签结果
        if e[4] == cls_val:
            cnt1 += 1
            if e[attr_index] == attr_val:
                cnt2 += 1
    return cnt2/cnt1

#利用后验概率计算先验概率
#参数：dataset-List列表，训练集，包含了样本数据和分类结果
#      test-List列表，测试样本属性列表
#      cls_y,cls_n-字符，分类标签Y，N
#返回值：{cls_y:PY,cls_n:PN}-浮点数，先验概率公式1-3的分子部分
def NB(dataset,test,cls_y,cls_n):
    PY = Prob(dataset,cls_y)
    PN = Prob(dataset,cls_n)
#对于测试样本test属性取值，计算其公式1-3的分子部分
#较大的值对应较大的分类可能
    for i, val in enumerate(test):
        PY *= ConditionP(dataset,cls_y,i,val)
        PN *= ConditionP(dataset,cls_n,i,val)
    return {cls_y:round(PY,6),cls_n:round(PN,6)}

print('理财产品促销客户1预测结果:')
BMVec = [2, 2, 3, 2]
prob=NB(BMData,BMVec,'Y','N')
print(prob)

print('理财产品促销客户2预测结果:')
BMVec = [1,3,1,1]
prob=NB(BMData,BMVec,'Y','N')
print(prob)

读入的数据文件首5行：
    客户ID  工龄  岗位性质  教育程度  婚姻状态 销售结果
0  10001   1     2     1     1    N
1  10002   1     2     3     1    N
2  10003   1     3     4     1    Y
3  10004   4     2     2     1    Y
4  10005   1     1     3     2    Y
转化后的2维列表前5项：
[[1, 2, 1, 1, 'N'], [1, 2, 3, 1, 'N'], [1, 3, 4, 1, 'Y'], [4, 2, 2, 1, 'Y']]
['工龄', '工作性质', '教育程度', '婚姻状况']
理财产品促销客户1预测结果:
{'N': 0.0, 'Y': 0.000943}
理财产品促销客户2预测结果:
{'N': 0.045776, 'Y': 0.002475}
